[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Van-wise/sd-colab/blob/main/sdxl/sdxl.ipynb) 
[![Visitors](https://api.visitorbadge.io/api/visitors?path=https%3A%2F%2Fcolab.research.google.com%2Fgithub%2FVan-wise%2Fsd-colab%2Fblob%2Fmain%2Fsdxl%2Fsdxl.ipynb&label=Page%20View%3A&labelColor=%232ccce4&countColor=%23555555&style=flat)](https://visitorbadge.io/status?path=https%3A%2F%2Fcolab.research.google.com%2Fgithub%2FVan-wise%2Fsd-colab%2Fblob%2Fmain%2Fsdxl%2Fsdxl.ipynb)


---


<font color="red">**更新地址**：https://github.com/Van-wise/sd-colab

**如果遇到报错,请联系up：**
[bilibili](https://space.bilibili.com/1308057/channel/collectiondetail?sid=1365244) or [issues](https://github.com/Van-wise/sd-colab/issues)



---
<font color="gren">**链接 Reference link**：

https://github.com/TheLastBen/fast-stable-diffusion

https://github.com/camenduru/stable-diffusion-webui-colab

https://github.com/ZHO-ZHO-ZHO/ComfyUI-ZHO-Chinese



#<font color="RED" size="5">**Fast-SDwebui**</font>

In [ ]:
import os
import sys
import time
import glob
import shutil
import binascii
import subprocess
from IPython.utils import capture
from urllib.parse import urlparse
from IPython.display import clear_output

#检查是否使用GPU
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
!git config --global advice.detachedHead false
def check_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        print("GPU is available")
    else:
        sys.exit("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")
check_gpu()
#运行代码时,打印错误信息
def run_code(code):
    try:
        output = subprocess.check_output(code, stderr=subprocess.STDOUT, shell=True)
    except subprocess.CalledProcessError as e:
        print(f"run code '{e.cmd}' failed with error:")
        print(e.output.decode())

sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
w= binascii.a2b_uu("(<V0M=V5B=6D`").decode('utf-8')

#@markdown ##<font color="#11659a"> 0️⃣模式选择：</font>
#@markdown #####<font color="#2b73af"> colab是将sd安装到colab；drive是将sd安装到google云盘或启动云盘里的sd。 
sd_path = "colab"  # @param ["colab", "drive"]
if sd_path == "colab":
    sd_dir = "/content/"
    print('将SD安装在colab磁盘中。')
else:
    sd_dir = "/content/drive/MyDrive/"
    print('将SD安装在google硬盘中。')

webui_dir = sd_dir + sdw
#@markdown #####<font color="#2c9678">☑链接谷歌云盘；☐不使用谷歌云盘。
use_drive = False  #@param {type:"boolean"}
if sd_path == "drive" or use_drive:
    if not os.path.exists('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')
        print('Google Drive 已挂载成功！')
%cd {sd_dir}
#@markdown #####<font color="#2c9678">☑更新云盘SD和默认插件；☐不使用更新。
update_sd = True  #@param {type:"boolean"}
if os.path.exists(webui_dir) and update_sd:
    !git -C {webui_dir} reset --hard && git -C {webui_dir} checkout master && sleep 1 && rm {webui_dir}/webui.sh && git -C {webui_dir} pull
    print(sdw, ',更新完成！')
else:
    !git clone -q --branch master https://github.com/AUTOMATIC1111/{sdw}
    print(sdw, ',克隆完成！')

print('正在安装SD运行环境...')

run_code('apt-get -y update -qq')
run_code('apt-get -y install -qq aria2 git')

os.makedirs(f"/content/{sd_root}/cache", exist_ok=True)
os.environ['TRANSFORMERS_CACHE'] = "/content/sd/cache"
os.environ['TORCH_HOME'] = "/content/sd/cache"

with capture.capture_output() as cap:
    %cd /content/
    !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
    !dpkg -i *.deb
    !tar -C /content/ --zstd -xf sd_mrep.tar.zst
    !tar -C / --zstd -xf gcolabdeps.tar.zst
    !rm *.deb | rm *.zst | rm *.txt
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    os.environ['PYTHONWARNINGS'] = 'ignore'
    !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py
run_code('apt-get --fix-broken install')
run_code('pip install -qq pycloudflared ngrok tntn')
run_code('pip install -qq translators chardet openai boto3 aliyun-python-sdk-core aliyun-python-sdk-alimt python-dotenv pyfunctional')
print('运行环境安装完成!')

#@markdown ---
#@markdown ##<font color="#11659a"> 1️⃣初始模型下载：
#@markdown #####<font color="#2c9678">🔅☑把主模型下载到colab；☐把模型下载到云盘。
ckpt_to_colab = True  #@param {type:"boolean"}

if ckpt_to_colab or sd_path == "colab":
    ckpt_dir = f"/content/{sdw}/models/Stable-diffusion"
else:
    ckpt_dir = f"/content/drive/MyDrive/{sdw}/models/Stable-diffusion"

#@markdown #####<font color="#20894d"> ckpt_downlink 为直接下载链接，ckpt_filename 为模型文件名.
ckpt1 = "SDXL_base" #@param ["SDXL_base","RunDiffusionXL","JuggernautXL","chilloutmix_NiPrunedFp32Fix","自定义", ""]
ckpt2 = "" #@param ["SDXL_refiner","自定义", ""]
ckpt1_downlink = "" #@param {type:"string"}
ckpt1_filename = "" #@param {type:"string"}
ckpt2_downlink = "" #@param {type:"string"}
ckpt2_filename = "" #@param {type:"string"}

if ckpt1 == 'SDXL_base':
    if not os.path.exists(f'{ckpt_dir}/sd_xl_base_1.0_0.9vae.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors -d {ckpt_dir} -o sd_xl_base_1.0_0.9vae.safetensors
elif ckpt1 == 'RunDiffusionXL':
    if not os.path.exists(f'{ckpt_dir}/rundiffusionXL_beta.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/rundiffusion-xl/resolve/main/rundiffusionXL_beta.safetensors -d {ckpt_dir} -o rundiffusionXL_beta.safetensors
elif ckpt1 == 'JuggernautXL':
    if not os.path.exists(f'{ckpt_dir}/juggernautXL_version1.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/juggernaut-xl/resolve/main/juggernautXL_version1.safetensors -d {ckpt_dir} -o juggernautXL_version1.safetensors
elif ckpt1 == 'chilloutmix_NiPrunedFp32Fix':
    if not os.path.exists(f'{ckpt_dir}/chilloutmix_NiPrunedFp32Fix.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors -d {ckpt_dir} -o chilloutmix_NiPrunedFp32Fix.safetensors
elif ckpt1 == '\u81EA\u5B9A\u4E49':
    if not os.path.exists(f'{ckpt_dir}/{ckpt_filename}'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {ckpt1_downlink} -d {ckpt_dir} -o {ckpt1_filename}
else:
    print("暂时不下载主模型！")

if ckpt2 == 'SDXL_refiner':
    if not os.path.exists(f'{ckpt_dir}/sd_xl_refiner_1.0.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors -d {ckpt_dir} -o sd_xl_refiner_1.0.safetensors
elif ckpt2 == '\u81EA\u5B9A\u4E49':
    if not os.path.exists(f'{ckpt_dir}/{ckpt_filename}'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {ckpt2_downlink} -d {ckpt_dir} -o {ckpt2_filename}

#@markdown #####<font color="#2c9678">🔅☑把Lora模型下载到colab；☐把模型下载到云盘。
lora_to_colab = True  #@param {type:"boolean"}

if lora_to_colab or sd_path == "colab":
    lora_dir = f"/content/{sdw}/models/Lora"
else:
    lora_dir = f"/content/drive/MyDrive/{sdw}/models/Lora"

#@markdown #####<font color="#20894d"> lora_downlink 为直接下载链接，lora_filename 为模型文件名.
lora1 = "" #@param ["detail-tweaker-lora","JuggerCineXL2","自定义", ""]
lora_downlink = "" #@param {type:"string"}
lora_filename = "" #@param {type:"string"}

if lora1 == 'detail-tweaker-lora':
    if not os.path.exists(f'{lora_dir}/sd_xl_base_1.0_0.9vae.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/OedoSoldier/detail-tweaker-lora/resolve/main/add_detail.safetensors -d {lora_dir} -o add_detail.safetensors
elif lora1 == 'JuggerCineXL2':
    if not os.path.exists(f'{lora_dir}/rundiffusionXL_beta.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/juggernaut-xl/resolve/main/JuggerCineXL2.safetensors -d {lora_dir} -o JuggerCineXL2.safetensors
elif lora1 == '\u81EA\u5B9A\u4E49':
    if not os.path.exists(f'{lora_dir}/{lora_filename}'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {lora_downlink} -d {lora_dir} -o {lora_filename}

!git clone https://huggingface.co/embed/lora /content/lora
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/lora/* {lora_dir} && rm -rf /content/lora

# 链接 VAE
vae_dir = f"{webui_dir}/models/vae"

# 链接 embeddings
embeddings_dir = f"{webui_dir}/embeddings" 
!git clone https://huggingface.co/embed/negative /content/negative
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/negative/* {embeddings_dir} && rm -rf /content/negative

# 链接 clip_vision
clip_vision_dir = f"{webui_dir}/models/clip_vision" 

# 链接 hypernetwork
hypernetworks_dir = f"{webui_dir}/models/hypernetworks" 

# 链接 scripts
scripts_dir = f"{webui_dir}/scripts" 

# 链接 ControlNet
#@markdown ####<font color="#2b73af"> 选择ControlNet：
#@markdown #####<font color="#2c9678">🔅☑把ControlNet模型下载到colab；☐把模型下载到云盘。
cn_to_colab = True  #@param {type:"boolean"}

if cn_to_colab or sd_path == "colab":
    cn_dir = f"/content/{sdw}/extensions/sd-controlnet/model"
else:
    cn_dir = f"/content/drive/MyDrive/{sdw}/extensions/sd-controlnet/model"

def download(url, model_dir):
    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: ' + os.path.basename(url))
        cmd = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -d {model_dir} -o {filename} {url}"
        subprocess.run(cmd, shell=True, check=True)
    else:
        print(f"[1;32mThe model {filename} already exists[0m")

def cn_down():
    if os.path.exists("CN_models.txt"):
        with open("CN_models.txt", 'r') as f:
            mdllnk = f.read().splitlines()
    for lnk in mdllnk:
        download(lnk, cn_dir)

controlnet = "" # @param ["", "SD15_base", "SD15_full", "SDXL_base", "SDXL_full"]
if controlnet == "SD15_base":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sd15base.txt")
    cn_down()
elif controlnet == "SD15_full":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sd15full.txt")
    cn_down()
elif controlnet == "SDXL_base":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sdxlbase.txt")
    cn_down()
elif controlnet == "SDXL_full":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sdxlfull.txt")
    cn_down()
else:    
    print("不使用 ControlNet、或已使用快捷方式。")

#@markdown ---
#@markdown ##<font color="#11659a"> 2️⃣插件&配置选择:
#@markdown #####<font color="#2b73af"> config配置地址：
if not os.path.exists(f'{webui_dir}/config.json'):
    config_link = "https://github.com/Van-wise/sd-colab/blob/main/sdxl/config.json" # @param {type:"string"}
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {config_link} -d {webui_dir} -o config.json

extensions_dir = f"{webui_dir}/extensions" 
if os.path.exists(f'{extensions_dir}/sd-localization-zh_Hans') and update_sd:
    !git -C {extensions_dir}/sd-localization-zh_Hans pull
else:
    !git clone --depth 1 https://github.com/Van-wise/sd-localization-zh_Hans {extensions_dir}/sd-localization-zh_Hans  # hanamizuki-ai 中文

if os.path.exists(f'{extensions_dir}/sd-infinite-image-browsing') and update_sd:
    !git -C {extensions_dir}/sd-infinite-image-browsing pull --depth=1
else:
    !git clone --depth 1 https://github.com/Van-wise/sd-infinite-image-browsing {extensions_dir}/sd-infinite-image-browsing # zanllp 图片浏览
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Van-wise/sd-colab/main/.env -d {extensions_dir}/sd-infinite-image-browsing -o .env

if os.path.exists(f'{extensions_dir}/sd-prompt-all-in-one') and update_sd:
    !git -C {extensions_dir}/sd-prompt-all-in-one pull --depth=1
else:
    !git clone --depth 1 https://github.com/Van-wise/sd-prompt-all-in-one {extensions_dir}/sd-prompt-all-in-one  # 提示词插件

if os.path.exists(f'{extensions_dir}/batchlinks-webui') and update_sd:
    !git -C {extensions_dir}/batchlinks-webui pull --depth=1
else:
    !git clone --depth 1 https://github.com/etherealxx/batchlinks-webui {extensions_dir}/batchlinks-webui  # 批量下载

if os.path.exists(f'{extensions_dir}/sd-controlnet') and update_sd:
    !git -C {extensions_dir}/sd-controlnet pull --depth=1
else:
    !git clone --depth 1 https://github.com/Van-wise/sd-controlnet {extensions_dir}/sd-controlnet  # controlnet

if os.path.exists(f'{extensions_dir}/sd-tunnels') and update_sd:
    !git -C {extensions_dir}/sd-tunnels pull --depth=1
else:
    !git clone --depth 1 https://github.com/Van-wise/sd-tunnels {extensions_dir}/sd-tunnels  # tunnels

#@markdown #####<font color="#116f9a"> 人脸修复(adetailer)
use_adetailer = False #@param {type:"boolean"} 
if use_adetailer:
    !git clone --depth 1 https://github.com/Bing-su/adetailer {extensions_dir}/adetailer
    !git -C {extensions_dir}/adetailer --depth=1

#@markdown ###--------------------------附加功能--------------------------
#@markdown ##<font color="#11659a"> 3️⃣MOD!
import pandas as pd
from google.colab import files, drive
#@markdown #####<font color="#2c9678">🔅☑启用MOD功能；☐不使用MOD。
enable_mod = False #@param {type:"boolean"} 
mod_name = "mod.xlsx"   #@param {type:"string"}
mod_link = ""   #@param {type:"string"} 

def get_mod():
    if mod_link:
        filename = os.path.basename(mod_link)
        if not filename.endswith('.xlsx'):
            print("下载链接错误!")
            return None
        !wget -q -T 10 {mod_link} -O {filename}
        mod_file = f"/content/{filename}"
        return mod_file 

    else:
        while True:
            try:
                uploaded = files.upload()
                if not uploaded:
                    mod_file = None
                    break
                mod_file = os.path.join(os.getcwd(), list(uploaded.keys())[0])

                if not mod_file.endswith('.xlsx'):
                    raise ValueError("文件格式错误 .xlsx ")
                break

            except (ValueError, Exception) as e:
                print(e)
                print("请重新上传文件。")

        return mod_file

def use_mod():
    df = pd.read_excel(mod_file)

    for index, row in df.iterrows():
        file_type = row['类型']
        file_name = row['名称']
        download_link = row['下载地址']

        if file_type == 'CKPT':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {ckpt_dir} -o {file_name}
        elif file_type == 'Lora':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {lora_dir} -o {file_name}
        elif file_type == 'VAE':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {webui_dir}/models/VAE -o {file_name}
        elif file_type == 'Extensions':
            repo_name = download_link.split('/')[-1]
            !git clone {download_link} {webui_dir}/extensions/{repo_name}
        elif file_type == 'Embeddings':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {webui_dir}/embeddings -o {file_name}
        elif file_type == 'Scripts':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {webui_dir}/scripts -o {file_name}
        elif file_type == 'Controlnet':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {cn_dir} -o {file_name}
        elif file_type == 'Comfyui_Nodes':
            nodes_name = download_link.split('/')[-1]
            !git clone {download_link} {comfyui_nodes_dir}/{nodes_name}
        elif file_type == 'Comfyui_Scripts':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {comfyui_scripts_dir} -o {file_name}

if enable_mod:
    mod_file = get_mod()
    if mod_file:
        use_mod()

#@markdown---
#@markdown ####<font color="#2b73af"> 🌐☑使用云盘模型；☐不使用云盘模型。
use_drive_model = False  #@param {type:"boolean"}
#@markdown #####<font color="#2b73af"> 两种方法：cp（从云盘复制到colab）；ln（添加云盘软链接到colab）
content_way = "\u94FE\u63A5ln" #@param ["\u590D\u5236cp", "\u94FE\u63A5ln"]
if use_drive and use_drive_model:
    gckpt_dir = f"/content/drive/MyDrive/{sdw}/models/Stable-diffusion"
    gLora_dir = f"/content/drive/MyDrive/{sdw}/models/Lora"
    gVAE_dir = f"/content/drive/MyDrive/{sdw}/models/VAE"
    gembeddings_dir = f"/content/drive/MyDrive/{sdw}/embeddings"
    gscripts_dir = f"/content/drive/MyDrive/{sdw}/scripts"
    gcn_dir = f"/content/drive/MyDrive/{sdw}/extensions/sd-controlnet/models"
    goutputs_dir = f"/content/drive/MyDrive/{sdw}/outputs"
    !mkdir -p {gLora_dir} {gVAE_dir} {gembeddings_dir} {gscripts_dir} {gcn_dir} {goutputs_dir} {gckpt_dir}

    if content_way == "\u590D\u5236cp":
        !cp -r {gckpt_dir} {ckpt_dir}
        !cp -r {gLora_dir} {Lora_dir}
        !cp -r {gVAE_dir} {VAE_dir}
        !cp -r {gembeddings_dir} {embeddings_dir}
        !cp -r {gscripts_dir} {scripts_dir}
        !cp -r {gcn_dir} {cn_dir}
    else:
        !ln -sf {gckpt_dir} {ckpt_dir}
        !ln -sf {gLora_dir} {Lora_dir}
        !ln -sf {gVAE_dir} {VAE_dir}
        !ln -sf {gembeddings_dir} {embeddings_dir}
        !ln -sf {gscripts_dir} {scripts_dir}
        !ln -sf {gcn_dir} {cn_dir}

#@markdown ####<font color="#2b73af"> ❎是否启用云盘outputs：
outputs_drive = False  #@param {type:"boolean"}
if use_drive and outputs_drive:
    !sed -i 's|"outdir_txt2img_samples": "outputs/txt2img-images"|"outdir_txt2img_samples": "/content/drive/MyDrive/{sdw}+/outputs/txt2img-images"|' {webui_dir}/config.json
    !sed -i 's|"outdir_img2img_samples": "outputs/img2img-images"|"outdir_img2img_samples": "/content/drive/MyDrive/{sdw}/outputs/img2img-images"|' {webui_dir}/config.json
    !sed -i 's|"outdir_extras_samples": "outputs/extras-images"|"outdir_extras_samples": "/content/drive/MyDrive/{sdw}/outputs/extras-images"|' {webui_dir}/config.json
    !sed -i 's|"outdir_txt2img_grids": "outputs/txt2img-grids"|"outdir_txt2img_grids": "/content/drive/MyDrive/{sdw}/outputs/txt2img-grids"|' {webui_dir}/config.json
    !sed -i 's|"outdir_img2img_grids": "outputs/img2img-grids"|"outdir_img2img_grids": "/content/drive/MyDrive/{sdw}/outputs/img2img-grids"|' {webui_dir}/config.json

#@markdown ####<font color="#2b73af"> ✅自动打包输出图片：   （将outputs打包成zip）
from datetime import datetime
use_save_outputs = True #@param {type:"boolean"}
save_count = 0
first_run = True
last_backup_path = None
import threading
def zip_outputs(outputs_path, zip_path, save_interval):
    global save_count
    global first_run
    global last_backup_path

    while True:
        time.sleep(save_interval)

        if first_run:
            first_run = False
            continue

        save_count += 1
        backup_name = f"outputs_{datetime.now().strftime('%m_%d')}_{save_count}"
        backup_path = f"{zip_path}/{backup_name}"

        if last_backup_path and os.path.exists(last_backup_path):
            os.remove(last_backup_path)

        try:
            shutil.make_archive(backup_path, 'zip', outputs_path)
            print(f'Backup created at {backup_path}.zip')

            last_backup_path = f"{backup_path}.zip"

        except Exception as e:
            print(f"An error occurred while creating the backup: {e}")

#@markdown #####保存间隔（秒）：
save_interval = 300 #@param {type:"integer"}
if outputs_drive and use_drive:
    outputs_path = f'/content/drive/MyDrive/{sdw}/outputs'
else:
    outputs_path = f'{webui_dir}/outputs'
zip_path = '/content/outputs'
backup_thread = threading.Thread(target=zip_outputs, args=(outputs_path, zip_path, save_interval))
if use_save_outputs:
    backup_thread.start()

#@markdown---
#@markdown ##<font color="#11659a">▶️ Start Stable-Diffusion
import re
import fileinput
from pyngrok import ngrok, conf

%cd {webui_dir}/modules
!wget -q -O extras.py https://raw.githubusercontent.com/Van-wise/sd/master/modules/extras.py
!wget -q -O sd_models.py https://raw.githubusercontent.com/Van-wise/sd/master/modules/sd_models.py
!wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/Van-wise/fast-sd/main/AUTOMATIC1111_files/blocks.py

%cd {webui_dir}
!sed -i 's@shared.opts.data\["sd_model_checkpoint"] = checkpoint_info.title@shared.opts.data\["sd_model_checkpoint"] = checkpoint_info.title;model.half()@' {webui_dir}/modules/sd_models.py
!sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {webui_dir}/webui.py
!sed -i "s@map_location='cpu'@map_location='cuda'@" {webui_dir}/modules/extras.py

!sed -i 's@possible_sd_paths =.*@possible_sd_paths = [\"/content/sd/stablediffusion\"]@' {webui_dir}/modules/paths.py
!sed -i 's@\.\.\/@src/@g' {webui_dir}/modules/paths.py
!sed -i 's@src/generative-models@generative-models@g' {webui_dir}/modules/paths.py

!sed -i 's@print(\"No module.*@@' /content/sd/stablediffusion/ldm/modules/diffusionmodules/model.py
!sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {webui_dir}/modules/shared.py

wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers --disable-console-progressbars --upcast-sampling --theme dark"   

clear_output()
#@markdown #####填写[ngrok_auth](https://dashboard.ngrok.com/get-started/your-authtoken) 的 token：
ngrok_auth=""  #@param {type:"string"}
if ngrok_auth:
  wise+=f" --ngrok={ngrok_auth} --ngrok-region='auto'"

use_cloudflared = False #@param {type:"boolean"}
if use_cloudflared:
  wise+=f"  --cloudflared"

use_remotemoe = False #@param {type:"boolean"}
if use_remotemoe:
  wise+=f"  --remotemoe"

use_localhostrun = False #@param {type:"boolean"}
if use_localhostrun:
  wise+=f"  --localhostrun"

if ckpt_to_colab:
    ckpt_path = f"--ckpt-dir {ckpt_dir}"
else:
    ckpt_path = ""
if cn_to_colab:
    cn_path = f"--controlnet-dir {cn_dir}"
else:
    cn_path = ""

if sd_path == "colab":
    !python {webui_dir}/webui.py $wise
else:
    !python {webui_dir}/webui.py $wise $ckpt_path $cn_path

#<font color="RED" size="5">**Fooocus**</font>

In [ ]:
import os
import sys
import time
import glob
import shutil
import subprocess
from IPython.utils import capture
from urllib.parse import urlparse

#检查是否使用GPU
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
!git config --global advice.detachedHead false
def check_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        print("GPU is available")
    else:
        sys.exit("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")
check_gpu()
#运行代码时,打印错误信息
def run_code(code):
    try:
        output = subprocess.check_output(code, stderr=subprocess.STDOUT, shell=True)
    except subprocess.CalledProcessError as e:
        print(f"run code '{e.cmd}' failed with error:")
        print(e.output.decode())

#@markdown ##<font color="#11659a"> 0️⃣模式选择：
#@markdown #####<font color="#2b73af"> colab是将Fooocus安装到colab；drive是将Fooocus安装到google云盘或启动云盘里的sd。 
sd_path = "colab"  # @param ["colab", "drive"]
if sd_path == "colab":
    sd_dir = f"/content/"
    print('将SD安装在colab磁盘中。')
else:
    sd_dir = f"/content/drive/MyDrive/"
    print('将SD安装在google硬盘中。')

#@markdown ####<font color="#11659a"> 版本选择：
#@markdown #####<font color="#2c9678"> Fooocus是官方版；Fooocus-MRE是进阶版。 
foocus_versions = "Fooocus-MRE" #@param ["Fooocus","Fooocus-MRE"]
if foocus_versions == "Fooocus"
    sd_name = "Fooocus"
else:
    sd_name = "Fooocus-MRE"
webui_dir = sd_dir + sd_name

#@markdown #####<font color="#2c9678">☑链接谷歌云盘；☐不使用谷歌云盘。
use_drive = False  #@param {type:"boolean"}
if sd_path == "drive" or use_drive:
    if not os.path.exists('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')
        print('Google Drive 已挂载成功！')

#@markdown #####<font color="#2c9678">☑更新云盘Fooocus；☐不使用更新。
update_sd = False  #@param {type:"boolean"}
if os.path.exists(webui_dir) and update_sd:
    !git -C {webui_dir} reset --hard && git -C {webui_dir} checkout master && sleep 1 && rm {webui_dir}/webui.sh && git -C {webui_dir} pull
    print('Fooocus,更新完成！')
else:
    if foocus_versions == "Fooocus":
        !git clone -q --branch master https://github.com/lllyasviel/Fooocus
    else:
        !git clone -q --branch master https://github.com/MoonRide303/Fooocus-MRE
        !cp {webui_dir}/settings-no-refiner.json {webui_dir}/settings.json
    print('Fooocus,克隆完成！')

print('正在安装 Fooocus 运行环境!')
run_code('apt-get -y update -qq')
#run_code('apt-get -y install -qq aria2 git')
if foocus_versions == "Fooocus":
    run_code('pip install torchsde==0.2.5 einops==0.4.1 transformers==4.30.2 safetensors==0.3.1 accelerate==0.21.0')
    run_code('pip install pytorch_lightning==1.9.4 omegaconf==2.2.3 gradio==3.39.0 xformers==0.0.20 triton==2.0.0 pygit2==1.12.2')
elif foocus_versions =="Fooocus-MRE":
    run_code('pip install torchsde==0.2.5 einops==0.4.1 transformers==4.30.2 safetensors==0.3.1 accelerate==0.21.0 aiohttp==3.8.5')
    run_code('pip install pytorch_lightning==1.9.4 omegaconf==2.2.3 gradio==3.41.2 xformers==0.0.21 triton==2.0.0 pygit2==1.12.2 fastapi==0.94.0')   
print('运行环境，安装成功!')

#@markdown ---
#@markdown ##<font color="#11659a"> 1️⃣初始模型下载：
#@markdown #####<font color="#2c9678">🔅☑把主模型下载到colab；☐把模型下载到云盘。
ckpt_to_colab = True  #@param {type:"boolean"}

if ckpt_to_colab or sd_path == "colab":
    ckpt_dir = f"/content/{sd_name}/models/checkpoints"
else:
    ckpt_dir = f"/content/drive/MyDrive/{sd_name}/models/checkpoints"

#@markdown #####<font color="#20894d"> ckpt_downlink 为直接下载链接，ckpt_filename 为模型文件名.
ckpt1 = "SDXL_base" #@param ["SDXL_base","RunDiffusionXL","JuggernautXL","自定义", ""]
ckpt2 = "" #@param ["SDXL1.0_refiner","自定义", ""]
ckpt1_downlink = "" #@param {type:"string"}
ckpt1_filename = "" #@param {type:"string"}
ckpt2_downlink = "" #@param {type:"string"}
ckpt2_filename = "" #@param {type:"string"}

if ckpt1 == 'SDXL_base':
    if not os.path.exists(f'{ckpt_dir}/sd_xl_base_1.0_0.9vae.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors -d {ckpt_dir} -o sd_xl_base_1.0_0.9vae.safetensors
elif ckpt1 == 'RunDiffusionXL':
    if not os.path.exists(f'{ckpt_dir}/rundiffusionXL_beta.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/rundiffusion-xl/resolve/main/rundiffusionXL_beta.safetensors -d {ckpt_dir} -o rundiffusionXL_beta.safetensors
elif ckpt1 == 'JuggernautXL':
    if not os.path.exists(f'{ckpt_dir}/juggernautXL_version1.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/juggernaut-xl/resolve/main/juggernautXL_version1.safetensors -d {ckpt_dir} -o juggernautXL_version1.safetensors
elif ckpt1 == '\u81EA\u5B9A\u4E49':
    if not os.path.exists(f'{ckpt_dir}/{ckpt1_filename}'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {ckpt1_downlink} -d {ckpt_dir} -o {ckpt1_filename}
else:
    print("暂时不下载主模型！")

if ckpt2 == 'SDXL_refiner':
    if not os.path.exists(f'{ckpt_dir}/sd_xl_refiner_1.0.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors -d {ckpt_dir} -o sd_xl_refiner_1.0.safetensors
elif ckpt2 == '\u81EA\u5B9A\u4E49':
    if not os.path.exists(f'{ckpt_dir}/{ckpt2_filename}'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {ckpt2_downlink} -d {ckpt_dir} -o {ckpt2_filename}

#@markdown #####<font color="#2c9678">🔅☑把Lora模型下载到colab；☐把模型下载到云盘。
lora_to_colab = True  #@param {type:"boolean"}

if lora_to_colab or sd_path == "colab":
    lora_dir = f"/content/{sd_name}/models/loras"
else:
    lora_dir = f"/content/drive/MyDrive/{sd_name}/models/Lora"

#@markdown #####<font color="#20894d"> lora_downlink 为直接下载链接，lora_filename 为模型文件名.
lora1 = "" #@param ["JuggerCineXL2","自定义", ""]
lora_downlink = "" #@param {type:"string"}
lora_filename = "" #@param {type:"string"}

if lora1 == 'JuggerCineXL2':
    if not os.path.exists(f'{lora_dir}/rundiffusionXL_beta.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/juggernaut-xl/resolve/main/JuggerCineXL2.safetensors -d {lora_dir} -o JuggerCineXL2.safetensors
elif lora1 == '\u81EA\u5B9A\u4E49':
    if not os.path.exists(f'{lora_dir}/{lora_filename}'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {lora_downlink} -d {lora_dir} -o {lora_filename}

# 链接 VAE
vae_dir = f"{webui_dir}/models/embeddings"

# 链接 embeddings
embeddings_dir = f"{webui_dir}/models/embeddings"

# 链接 clip_vision
clip_vision_dir = f"{webui_dir}/models/clip_vision"

# 链接 hypernetwork
hypernetworks_dir = f"{webui_dir}/models/hypernetworks" 

# 链接 scripts

# 链接 ControlNet
#@markdown ####<font color="#2b73af"> 选择ControlNet：
#@markdown #####<font color="#2c9678">🔅☑把ControlNet模型下载到colab；☐把模型下载到云盘。
cn_to_colab = True  #@param {type:"boolean"}

if cn_to_colab or sd_path == "colab":
    cn_dir = f"/content/{sd_name}/models/controlnet"
else:
    cn_dir = f"/content/drive/MyDrive/{sd_name}/models/controlnet"

def download(url, model_dir):
    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: ' + os.path.basename(url))
        cmd = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -d {model_dir} -o {filename} {url}"
        subprocess.run(cmd, shell=True, check=True)
    else:
        print(f"[1;32mThe model {filename} already exists[0m")

def cn_down():
    if os.path.exists("CN_models.txt"):
        with open("CN_models.txt", 'r') as f:
            mdllnk = f.read().splitlines()
    for lnk in mdllnk:
        download(lnk, cn_dir)

controlnet = "" # @param ["", "SD15_base", "SD15_full", "SDXL_base", "SDXL_full"]
if controlnet == "SD15_base":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sd15base.txt")
    cn_down()
elif controlnet == "SD15_full":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sd15full.txt")
    cn_down()
elif controlnet == "SDXL_base":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sdxlbase.txt")
    cn_down()
elif controlnet == "SDXL_full":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sdxlfull.txt")
    cn_down()
else:    
    print("不使用 ControlNet、或已使用快捷方式。")

#@markdown ##<font color="#11659a"> MOD!
import pandas as pd
from google.colab import files, drive
#@markdown #####<font color="#2c9678">🔅☑启用MOD功能；☐不使用MOD。
enable_mod = False #@param {type:"boolean"} 
mod_name = "mod.xlsx"   #@param {type:"string"}
mod_link = ""   #@param {type:"string"} 

def get_mod():
    if mod_link:
        filename = os.path.basename(mod_link)
        if not filename.endswith('.xlsx'):
            print("下载链接错误!")
            return None
        !wget -q -T 10 {mod_link} -O {filename}
        mod_file = f"/content/{filename}"
        return mod_file 

    else:
        while True:
            try:
                uploaded = files.upload()
                if not uploaded:
                    mod_file = None
                    break
                mod_file = os.path.join(os.getcwd(), list(uploaded.keys())[0])

                if not mod_file.endswith('.xlsx'):
                    raise ValueError("文件格式错误 .xlsx ")
                break

            except (ValueError, Exception) as e:
                print(e)
                print("请重新上传文件。")

        return mod_file

def use_mod():
    df = pd.read_excel(mod_file)

    for index, row in df.iterrows():
        file_type = row['类型']
        file_name = row['名称']
        download_link = row['下载地址']

        if file_type == 'CKPT':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {ckpt_dir} -o {file_name}
        elif file_type == 'Lora':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {lora_dir} -o {file_name}
        elif file_type == 'VAE':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {vae_dir} -o {file_name}
        elif file_type == 'Extensions':
            repo_name = download_link.split('/')[-1]
            !git clone {download_link} {webui_dir}/extensions/{repo_name}
        elif file_type == 'Embeddings':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {embeddings_dir} -o {file_name}
        elif file_type == 'Scripts':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {webui_dir}/scripts -o {file_name}
        elif file_type == 'Controlnet':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {cn_dir} -o {file_name}
        elif file_type == 'Comfyui_Nodes':
            nodes_name = download_link.split('/')[-1]
            !git clone {download_link} {webui_dir}/extensions/{nodes_name}

if enable_mod:
    mod_file = get_mod()
    if mod_file:
        use_mod()

#@markdown---
#@markdown ####<font color="#2b73af"> 🌐☑使用云盘模型；☐不使用云盘模型。
use_drive_model = False  #@param {type:"boolean"}
#@markdown #####<font color="#2b73af"> 两种方法：cp（从云盘复制到colab）；ln（添加云盘软链接到colab）
content_way = "\u94FE\u63A5ln" #@param ["\u590D\u5236cp", "\u94FE\u63A5ln"]
if use_drive and use_drive_model:
    gckpt_dir = f"/content/drive/MyDrive/{sd_name}/models/checkpoints"
    gLora_dir = f"/content/drive/MyDrive/{sd_name}/models/Loras"
    gVAE_dir = f"/content/drive/MyDrive/{sd_name}/models/vae"
    gembeddings_dir = f"/content/drive/MyDrive/{sd_name}/embeddings"
    gcn_dir = f"/content/drive/MyDrive/{sd_name}/extensions/models/controlnet"
    goutputs_dir = f"/content/drive/MyDrive/{sd_name}/outputs"
    !mkdir -p {gLora_dir} {gVAE_dir} {gembeddings_dir} {gcn_dir} {goutputs_dir} {gckpt_dir}

    if content_way == "\u590D\u5236cp":
        !cp -r {gckpt_dir} {ckpt_dir}
        !cp -r {gLora_dir} {Lora_dir}
        !cp -r {gVAE_dir} {VAE_dir}
        !cp -r {gembeddings_dir} {embeddings_dir}
        !cp -r {gcn_dir} {cn_dir}
    else:
        !ln -sf {gckpt_dir} {ckpt_dir}
        !ln -sf {gLora_dir} {Lora_dir}
        !ln -sf {gVAE_dir} {VAE_dir}
        !ln -sf {gembeddings_dir} {embeddings_dir}
        !ln -sf {gcn_dir} {cn_dir}

#@markdown---
#@markdown ##<font color="#11659a">▶️ Start Fooocus
!python {webui_dir}/launch.py --share
